# Visual Q&A Demo

## Setup
1-Installs required packages   
2-Downloads sample data and a pretrained model.     

In [1]:
using Pkg; Pkg.activate("."); Pkg.instantiate(); #install required packages
include("demosetup.jl") 
include("src/main.jl")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


┌ Warning: Some registries failed to update:
│     — /home/gridsan/eakyurek/.julia/registries/General — failed to fetch from repo
└ @ Pkg.Types /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Pkg/src/Types.jl:1269


Demo setup is completed


scalepixel (generic function with 1 method)

## Initialization
1-Loads the sample demo data (image features,questions,vocabulary).   
    * w2i : words to one-hot index   : w2i["cube"] = 30  
    * a2i : answer to one-hot index  : a2i["gray"] = 8
    * i2w : one-hot index to words   : i2w[2] = "Are"   
    * i2a : one-hot index to answers : i2a[5] = "large"   
2-Loads the pretrained model, and its hyper-parameters `o`.     

In [2]:
feats,qstsns,(w2i,a2i,i2w,i2a) = loadDemoData("data/demo/");
_,Mrun,o = loadmodel("models/macnetnew.jld2";onlywrun=true);
value(params(Mrun)[1]) isa Array ? KnetLayers.settype!(Array) : KnetLayers.settype!(KnetArray)

Loading demo features ...
Loading demo questions ...
Loading dictionaries ...


Array{Float32,N} where N

## Sample Data
1-Randomly selects (question,image) pair from the sample data   
2-Make predictions for the question and checks whether the prediction is correct   

In [9]:
rnd        = rand(1:length(qstsns)) # try 41
inst       = qstsns[rnd]
feat       = arrtype(feats[:,:,:,rnd:rnd])
question   = Array{Int}(inst[2])
answer     = inst[3];
family     = inst[4];
results,prediction,interoutputs = singlerun(Mrun,feat,question;p=o[:p],selfattn=o[:selfattn],gating=o[:gating]);
interoutputs = first.(interoutputs)
answer==prediction[1]

false

In [12]:
interoutputs,prediction[1],answer

([6, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28], 28, 12)

In [ ]:
img = load("data/demo/CLEVR_v1.0/images/val/$(inst[1])")

In [ ]:
textq  = i2w[question];
println("Question: ",join(textq," "))
texta  = i2a[answer];
println("Answer: $(texta)\nPrediction: $(i2a[prediction]) ")

## User Data
You can enter your own question about the image and test whether the prediction is correct

In [ ]:
userinput = readline(stdin)
words = split(userinput) # tokenize(userinput)
question = [get!(w2i,wr,1) for wr in words]
results,prediction = singlerun(Mrun,feat,question;p=o[:p],selfattn=o[:selfattn],gating=o[:gating]);
println("Question: $(join(i2w[question]," "))")
println("Prediction: $(i2a[prediction])")

## Visualize
`visualize` function visualizes attention maps for each time step of the mac network

In [ ]:
visualize(img,results;p=o[:p])